# Scripts de mosaicos

In [1]:
import ee
import geemap
import os
import folium

## Área de Estudio

In [2]:
Map = geemap.Map()

In [3]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

### Patagonia

In [4]:
work_dir = os.path.expanduser('../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')

Map(bottom=11376.0, center=[-53.38332836757155, -72.19116210937501], controls=(WidgetControl(options=['positio…

## Funciones

In [8]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [9]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [10]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [11]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [12]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [13]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [14]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [15]:
def Cloud(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30))  

In [16]:
def Agua(image):
    return image.updateMask(ndwi.gte(-0.1))

In [99]:
def Ndvi1(image):
    return image.updateMask(ndvi.lt(0.2))

In [17]:
def Ndvi2(image):
    return image.updateMask(ndvi.gte(0.2))

## Generación de Colecciones

In [ ]:
start_date = '2020-10-01'
end_date = '2021-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [ ]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [73]:
coleccion = l8.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [ ]:
Map.addLayer(l8, vizParams, 'false color composite')
Map

### Greenest

In [74]:
greenest = coleccion.qualityMosaic('NDVI')

In [75]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(greenest, visParams, 'comp mas verde')
Map.centerObject(patagonia_grande, 6)
Map

Map(bottom=5699.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position…

In [ ]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [ ]:
image_rgb = greenest.visualize(**{
    'bands' : ['B5','B4','B3'],
    'max':0.5
})

In [ ]:
Map.addLayer(image_rgb, {}, 'test visualizacion')

#### Máscara

In [83]:
coleccion2 = coleccion.map(Cloud)
greenest2 = coleccion2.qualityMosaic('NDVI')
map_sin_nubes = folium.Map(location = [-45, -73], zoom_start = 5)
map_sin_nubes.add_ee_layer(greenest2, vizParams, 'sin nubes')
display(map_sin_nubes)

In [108]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 7)
map_ndwi_mask.add_ee_layer(ndwi_mask, ndwi_viz, 'NDWI sin tierra')
display(map_ndwi_mask)

#### Test

In [88]:
coleccion2 = coleccion.map(Cloud)
coleccion2 = coleccion2.map(Ndvi2)

greenest2 = coleccion2.qualityMosaic('NDVI')

In [89]:
map_sin_nubes = folium.Map(location = [-45, -73], zoom_start = 7)
map_sin_nubes.add_ee_layer(greenest2, vizParams, 'sin nubes')
display(map_sin_nubes)

In [104]:
greenest2 = greenest2.visualize(**{
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
})


### Agua

In [66]:
start_date_agua = '2020-11-01'
end_date_agua = '2021-03-31'

l8_agua = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date_agua, end_date_agua)


In [67]:
coleccion_agua = l8_agua.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [68]:
aquast = coleccion_agua.qualityMosaic('NDWI')

In [69]:
visParams = {
  'bands': ['B4', 'B3', 'B1'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(aquast, visParams, 'comp mas agua')
Map.centerObject(patagonia_grande, 6)
Map

Map(bottom=22393.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['positio…

In [26]:
ndwi = greenest.select('NDWI')
ndwi_viz = {'min': -1, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.addLayer(ndwi, ndwi_viz, 'NDWI')

In [27]:
ndwi_rgb = ndwi.visualize(**{
    'min': 0.5,
    'max':1,
    'palette': ['00FFFF', '0000FF']
})

#### Máscara

In [86]:
ndwi_mask = ndwi.updateMask(ndwi.gte(-0.1)) # Máscara partes sin agua, NDWI < -0.1

In [87]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask.add_ee_layer(ndwi_mask, ndwi_viz, 'NDWI sin tierra')
display(map_ndwi_mask)

In [100]:
coleccion3 = coleccion.map(Cloud)
coleccion3 = coleccion3.map(Ndvi1)
aquast2 = coleccion3.qualityMosaic('NDWI')

In [101]:
map_sin_tierra = folium.Map(location = [-45, -73], zoom_start = 7)
map_sin_tierra.add_ee_layer(aquast2, vizParams, 'sin tierra')
display(map_sin_tierra)

### Tierra

In [ ]:
ndgi = greenest.select('NDGI')
ndgi_viz = {'min': -1, 'max': 1, 'palette': ['FF00FF', 'FFFF00']}
Map.addLayer(ndgi, ndgi_viz, 'NDGI')

In [41]:
start_date = '2020-01-01'
end_date = '2021-03-31'

l8tierra = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [42]:
coleccion_tierra = l8tierra.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [43]:
tierrast = coleccion_tierra.qualityMosaic('NDGI')

In [44]:
Map.addLayer(tierrast, vizParams, 'test tierra')
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position'], widget=HBox…

## Mosaicos

### RGB

In [ ]:
aquast_rgb = aquast.visualize

In [105]:
mosaic = ee.ImageCollection([aquast, greenest]).mosaic()

In [106]:
Map.addLayer(mosaic, vizParams, 'Mosaico')
Map

Map(bottom=11241.0, center=[-51.57706953722566, -72.01538085937501], controls=(WidgetControl(options=['positio…

In [107]:
map_mosaic = folium.Map(location = [-45, -73], zoom_start = 7)
map_mosaic.add_ee_layer(mosaic, vizParams, 'mosaic')
display(map_mosaic)

In [76]:
mosaic2 = ee.ImageCollection([aquast2, greenest2]).mosaic()

In [77]:
mosaic2.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy',
 'NDWI',
 'NDGI']

In [78]:
Map.addLayer(mosaic2, vizParams, 'Mosaico2')
Map

Map(bottom=22481.0, center=[-53.57293832648609, -69.99938964843751], controls=(WidgetControl(options=['positio…

In [48]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position'], widget=HBox…

## Clasificación Supervisada

In [49]:
work_dir = os.path.expanduser('../Data/shapes/')

val_shp= os.path.join(work_dir, 'validacion.shp')
merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [50]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [51]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [52]:
dataset = mosaic2.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [53]:
datasetval = mosaic2.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [54]:
print(dataset.size().getInfo())

EEException: Expected a homogeneous image collection, but an image with incompatible bands was encountered:
  First image type: 18 bands ([B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, BQA, NDVI, date, month, doy, NDWI, NDGI]).
Current image type: 3 bands ([vis-red, vis-green, vis-blue]).
          Image ID: 2
Some bands might require explicit casts.

In [ ]:
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'NDWI',
 'NDGI']

label = 'ID'

In [ ]:
randomforest = ee.Classifier.smileRandomForest(100).train(dataset, label, bands)
resultRF = mosaic2.select(bands).classify(randomforest)

In [ ]:

class_values1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

class_palette1 = ['22c129', # 1
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [ ]:
resultRF = resultRF.set('classification_class_values', class_values1)
resultRF = resultRF.set('classification_class_palette', class_palette1)

In [ ]:
Map.addLayer(resultRF, {}, 'Land cover clasificado RF 100' )
legend_dict = {
    '01 Achaparrado': '22c129',
    '02 Agrícola': 'f0ff00',
    '03 Agua': '0c40f1',
    '04 Arenal o Roca': '685205',
    '05 Bosque Degradado': '0d4404',
    '06 Cuncunas': 'e82d13',
    '07 Estepa': 'f99504',
    '08 Humedal': '17d4c6',
    '09 Mallin': '1c6330',
    '10 Matorral abierto': '24ff00',
    '11 Matorral denso': '2de50f',
    '12 Plantaciones': 'ccba7d',
    '13 Praderas': 'dec61a',
    '14 Primario': '28d909',
    '15 Ríos': '03c2ff',
    '16 Secundario': '4e8c44',
    '17 Sombra topo': 'fdfdfd',
    '18 Suelo desnudo': 'c0c233'
}
Map.add_legend(legend_title="Random Forest", legend_dict=legend_dict)
Map